In [1]:

from instancePDSVRP import instancePDSVRP
import copy
import time
import heuristic as h
import utilities as u

Method that returns also the initial solution

In [2]:
def SISSRs_with_initial_solution_return(instance, sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma, n_nearest, delta, epsilon, iter_imp, iter_max, p_min, p_max, max_unfeasible_swaps_perturb):
    s_0 = h.initial_solution_construction(instance, w1, w2, w3, w4, w5, gamma, n_nearest)
    s_curr = s_0
    s_best = s_0
    iterations_without_improvement = 0
    iteration_counter = 0
    while (iteration_counter < iter_max):
        s = h.ruin_and_recreate(instance, copy.deepcopy(s_curr), sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma)
        if u.cost(instance, s) < u.cost(instance, s_curr)*(1+delta):
            s_curr = h.local_search(instance, s, n_nearest)
            if u.cost(instance, s_curr) < u.cost(instance, s_best):
                s_best = s_curr
                iterations_without_improvement = 0
            else:
                iterations_without_improvement += 1
        if iterations_without_improvement >= iter_imp:
            s_curr = h.perturbate(instance, s_curr, p_min, p_max, max_unfeasible_swaps_perturb)
            iterations_without_improvement = 0
        delta = delta * epsilon
        iteration_counter+=1

    return s_0, s_best

Medium instances test

In [3]:
def test_robustness(repetitions, heuristic, instances,  sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma, n_nearest, delta, epsilon, iter_imp, iter_max, p_min, p_max, max_unfeasible_swaps_perturb):
    results = []
            
    for instance_name in instances:
        instance = instancePDSVRP("instances/" + instance_name + ".txt")

        print("Instance: ", instance_name)
        avarage_cost = 0
        avarage_initial_cost = 0
        best_cost = None
        avarage_time = 0

        customers_served_by_drones = 0
        trucks_used = 0

        for i in range(repetitions):
            print("repetition ", i+1)
    
            start_time = time.time()
            init_sol, sol = heuristic(instance, sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma, n_nearest, delta, epsilon, iter_imp, iter_max, p_min, p_max, max_unfeasible_swaps_perturb)
            end_time = time.time()
            cost_heu = u.cost(instance, sol)
            cost_init_heu = u.cost(instance, init_sol)
            elapsed_time_heuristic = end_time - start_time
            avarage_time += elapsed_time_heuristic
            avarage_cost += cost_heu
            avarage_initial_cost += cost_init_heu

            print("costo trovato alla ripetizione ", i + 1, ":", cost_heu)

            if (best_cost == None or (best_cost != None and best_cost > cost_heu)):
                best_cost = cost_heu
                customers_served_by_drones = 0
                trucks_used = 0
                for drone_tour in sol[0][1]:
                    customers_served_by_drones += len(drone_tour)
                for truck_tour in sol[0][0]:
                    if len(truck_tour) > 0:
                        trucks_used += 1
                
        avarage_time = avarage_time / repetitions
        avarage_cost = avarage_cost / repetitions
        avarage_initial_cost = avarage_initial_cost / repetitions

        cost_difference = ((avarage_cost - best_cost) / best_cost) * 100
        initial_cost_difference = ((avarage_initial_cost - best_cost) / best_cost) * 100

        N_t = [i for i in range(1,instance.N) if (instance.w[i] <=  instance.Q_d and ((instance.t_d[i] * 2) <= instance.d_end))]
        drone_eligible_customers = len(N_t)

        results.append({
            "instance": instance_name,
            "best_cost": best_cost,
            "avarage_gap": cost_difference,
            "avarage_gap_initial_solution": initial_cost_difference,               
            "avarage_time": avarage_time,
            "drone_eligible_customers": drone_eligible_customers,
            "customers_served_by_drones": customers_served_by_drones,
            "trucks_used" : trucks_used,
            "repetitions": repetitions,
            "iter_imp": iter_imp,
            "iter_max": iter_max
        })
    
    return results

In [ ]:
heuristic = SISSRs_with_initial_solution_return
instances = []

for customers_position in ["c", "r", "rc"]:
    for depot_position in ["c", "e", "r"]:
        instances.append("50-" + customers_position + "-1-" + depot_position)

#instances = ["small_instances/10-c-1-c"]

results = test_robustness(10, heuristic, instances, 0.3, 7.5, 7.5, 5,1,1,2,2, 0.1, 20, 0.1, 0.999975, 100, 1000, 3, 5, 9)

u.save_results_to_csv(results, "results/medium_instances_results.csv")